## Task 4 - Web Mining

For an e-commerce business, the website structure and site plan were established
with the efficiency and usability in mind, but its effectiveness was not verified. Only
basic statistics have been produced through simple report and query techniques,
but they provide no means for sophisticated web site analysis and predictions. Your
task is to determine the user browsing patterns of the website and analyse those
patterns to provide recommendations to improve the website.
You have been provided with a raw log file, WEBLOG. This is the original text file
that needs to be processed with the steps required for web usage mining as
explained in the practical. Detail of the data set is given below.

Your task is to pre-process the given dataset and apply a suitable data mining
operation, such as classification or clustering or association mining, to the raw
WEBLOG data set. Answer the followings in relation to this data and the analyses
that you have chosen.

1. Pre-process the WEBLOG data to remove any unproductive items from the
log file such as graphics, sound, etc and also identify UserID, SessionID and
STEPS (within a given session, the order of the pages visited) based on IP
address, date and time.
**See Below**
2. What variables did you include in the analysis and what were their roles and
measurement level set? Justify your choice.
***TODO***
    * not ip
    * not status
3. Apply a datamining task on the processed dataset. Explain the rationale
behind selecting the data mining task/method.
***TODO***
4. Discuss the results obtained. Discuss also the applicability of findings of the
method. You should include only a high-level managerial kind of discussion
on the findings. It should not just be an interpretation of results as shown in
results.
***TODO***

In [1]:
import pandas as pd

df = pd.read_csv('datasets/Weblog.csv')

df.head()

,IP address,Timestamp,Request,Staus
0,10.128.2.1,[29/Nov/2017:06:58:55,GET /login.php HTTP/1.1,200
1,10.128.2.1,[29/Nov/2017:06:59:02,POST /process.php HTTP/1.1,302
2,10.128.2.1,[29/Nov/2017:06:59:03,GET /home.php HTTP/1.1,200
3,10.131.2.1,[29/Nov/2017:06:59:04,GET /js/vendor/moment.min.js HTTP/1.1,200
4,10.130.2.1,[29/Nov/2017:06:59:06,GET /bootstrap-3.3.7/js/bootstrap.js HTTP/1.1,200


## Data Cleaning

In [2]:
df.rename(columns= {'Staus': 'Status', 'IP address': 'IP'}, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15789 entries, 0 to 15788
Data columns (total 4 columns):
IP           15789 non-null object
Timestamp    15789 non-null object
Request      15789 non-null object
Status       15789 non-null int64
dtypes: int64(1), object(3)
memory usage: 493.5+ KB


In [3]:
# correct date datatype
df['Datetime'] = pd.to_datetime( df.Timestamp, format='[%d/%b/%Y:%H:%M:%S')
df.drop(['Timestamp'], axis=1, inplace=True)

In [4]:
# remove all HTTP1.1 values and GET/POST values
df.Request.replace(regex=True, inplace=True, to_replace=r' HTTP/1\.1', value=r'')
df.Request.replace(regex=True, inplace=True, to_replace=r'GET ', value=r'')
df.Request.replace(regex=True, inplace=True, to_replace=r'POST ', value=r'')

In [5]:
# mask to filter all .js, .css, fonts and .png files files
mask = (df.Request.str.endswith('.js') | df.Request.str.endswith('.css') \
        | df.Request.str.startswith('/fonts') | df.Request.str.startswith('/img'))
print ("# Rows before: ", len(df))

# keep records not in mask
df = df[~mask]

print("Rows after: ", len(df))

# filter !OK all requests
df = df[df['Status'] == 200]
print("Rows after success filter: ", len(df))

# Rows before:  15789
Rows after:  10607
Rows after success filter:  6865


In [6]:
df.Request.value_counts()

/login.php                                                         3279
/home.php                                                           485
/contestproblem.php?name=RUET%20OJ%20Server%20Testing%20Contest     393
/contest.php                                                        240
/archive.php                                                        239
/standings.php?id=16                                                148
/allsubmission.php                                                  140
/login.php?value=fail                                               128
/sign.php                                                           124
/                                                                   121
/compile.php                                                         95
/compiler.php                                                        90
/contestsubmission.php?id=16                                         79
/pcompile.php                                                   

## User ID

In [8]:
from collections import defaultdict
import datetime

df2 = df.copy() # backup dataset

# sort rows by descending datetime
df.sort_values(by='Datetime', inplace=True)

# iniate sessionID and userID to 0
session_id = 0
user_id = 0

# create dicts to hold last access information
last_access = defaultdict(lambda:datetime.datetime.utcfromtimestamp(0))

# dict to find previous session, user ID and steps assigned to specific date/ip
session_dict = defaultdict(lambda:1)
user_id_dict = defaultdict(lambda:1)
session_steps = defaultdict(lambda:1)

# function to be applies row wise
# for each row, produce session, user ID and path traversal
def get_log_user_info(row):
    # access global variables shared between all rows
    global session_id, user_id, session_dict, user_id_dict, session_steps, last_access
    
    session_key = str(row['Datetime'].date()) + '_' + row['IP'] # date + IP key for find
    user_key = str(row['Datetime'].date()) + '_' + row['IP'] #+ '_' + row['Agent'] # date
    time_diff_session = row['Datetime'] - last_access[session_key] # session time diff
    time_diff_user = row['Datetime'] - last_access[user_key] # user time diffme diff
    
    # if the time diff from previous session is > 30 mins, assign new session ID
    if time_diff_session.total_seconds() > 1800:
        session_id += 1
        session_dict[session_key] = session_id
    # if the time diff from previous session is > 60 mins, assign new user ID
    if time_diff_user.total_seconds() > 3600:
        user_id += 1
        user_id_dict[user_key] = user_id
    # update last access for session and user
    last_access[session_key] = row['Datetime']
    last_access[user_key] = row['Datetime']
    
    # assign extracted info from the row
    row['SessionId'] = session_dict[session_key]
    row['Step'] = session_steps[row['SessionId']]
    row['UserId'] = user_id_dict[user_key]
    session_steps[row['SessionId']] += 1
    
    return row

# appy function above to get a new df with added info
df = df.apply(get_log_user_info, axis=1)

In [9]:
df.head()

,IP,Request,Status,Datetime,SessionId,Step,UserId
4312,10.130.2.1,/,200,2017-11-07 23:59:19,1,2,1
5430,10.130.2.1,/,200,2017-11-07 23:59:19,1,3,1
5432,10.129.2.1,/login.php,200,2017-11-08 00:39:07,2,1,2
4314,10.129.2.1,/login.php,200,2017-11-08 00:39:07,2,2,2
5434,10.129.2.1,/login.php,200,2017-11-08 01:36:22,3,1,2
